# 0. what for
1. 使用全部的data config,但是是部分的data record，
    - 相当于mask掉20%的(n,p)值，对这些做predict
2. GT-2是用SA的值吗？
    - 我建议用SA的方法，这样才能突出InferNet的作用以及game theory model的limitation
# 1. Preparation
1. metric是MSE or MLE?

In [3]:
import random
import pickle
import time

import torch.utils.data
import torchvision.transforms as transforms
import visdom

from mydataset import myDataset_mask
import torch.nn as nn
import my_collate_fn
from my_collate_fn import my_collate_fn_mask
from torch.utils.data import DataLoader, SubsetRandomSampler
# from tensorboardX import SummaryWriter
from importlib import reload
from torch.nn.utils import clip_grad_norm_
from plot import draw_loss
from plot import draw_metric
from models import Conv_1_4

######### Ray Tune

import loss
import plot
# import models

reload(loss)    # 必须reload！！
reload(plot)
reload(my_collate_fn)
# reload(models)

from Config.config_MDN_mask import DefaultConfig
from loss import loss_fn_wei
from loss import loss_fn_mse
from loss import validate

opt = DefaultConfig()

- 得到某个seed下，dataset中可以mask的(n,p)的idx

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [6]:
dataset = myDataset_mask(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.data_key_path, opt.NLL_metric_path, opt.val_mask_idx_path, opt.test_mask_idx_path)


In [7]:
mlp = Conv_1_4(opt.N_gaussians).to(device)
mlp = mlp.to(device=device)

############ learning rate strategy ############
# 1.
# Set different lr for params
# id: id() 函数返回对象的唯一标识符，标识符是一个整数。返回对象的内存地址。
shape_params = list(map(id, mlp.z_shape.parameters()))
scale_params = list(map(id, mlp.z_scale.parameters()))
pi_params = list(map(id, mlp.z_pi.parameters()))


params_id = shape_params + scale_params + pi_params

base_params = filter(lambda p: id(p) not in params_id, mlp.parameters())
params = [{'params': base_params},         # 如果对某个参数不指定学习率，就使用最外层的默认学习率
          {'params': mlp.z_pi.parameters(), 'lr': opt.lr_for_pi},
        {'params': mlp.z_shape.parameters(), 'lr': opt.lr_for_shape},
          {'params': mlp.z_scale.parameters(), 'lr': opt.lr_for_scale}]

optimizer = torch.optim.AdamW(params, lr=1e-3,weight_decay=opt.weight_decay)

# 2.
# Or set lr decay
# StepLR为步进，每step_size个epoch，lr*gamma
scheduler  = torch.optim.lr_scheduler.StepLR(optimizer,step_size=15,gamma=opt.StepLR_gamma) # opt.StepLR_step_size
